In [ ]:
import argparse
import itertools
import os
import pickle
from collections.abc import Generator
from dataclasses import dataclass, field
from datetime import datetime
from typing import Any, List, Optional, Tuple, Union

import numpy as np
import numpy.typing as npt
import torch
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
from scipy.stats import ortho_group
from torchtyping import TensorType
from tqdm import tqdm

In [ ]:
@dataclass
class ToyArgs:
    device: str = "cuda:0" if torch.cuda.is_available() else "cpu"
    tied_ae: bool = False
    seed: int = 2
    learned_dict_ratio: float = 1.0
    output_folder: str = "outputs"
    # dtype: torch.dtype = torch.float32
    activation_dim: int = 32
    feature_prob_decay: float = 0.99
    feature_num_nonzero: int = 8
    correlated_components: bool = False
    n_ground_truth_components: int = 32
    batch_size: int = 40_000
    lr: float = 2e-3
    epochs: int = 5_000  # 50_000
    n_components_dictionary: int = 64
    n_components_dictionary_trans: int = 64
    l1_alpha: float = 5e-3
    lp_alphas: list[float] = field(
        default_factory=lambda: [
            1e-6,
            3e-6,
            1e-5,
            3e-5,
            1e-4,
            3e-4,
            1e-3,
            3e-3,
            1e-2,
            3e-2,
            1e-1,
            3e-1,
            1,
            3,
        ]
    )
    p: float = 0.5  # the p in the L_p norm used to induce sparsity
    ps: list[float] = field(
        default_factory=lambda: [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1]
    )

# Define data generators and autoencoders

In [ ]:
@dataclass
class RandomDatasetGenerator(Generator):
    activation_dim: int
    n_ground_truth_components: int
    batch_size: int
    feature_num_nonzero: int
    feature_prob_decay: float
    correlated: bool
    device: Union[torch.device, str]

    feats: Optional[TensorType["n_ground_truth_components", "activation_dim"]] = None
    generated_so_far: int = 0

    frac_nonzero: float = field(init=False)
    decay: TensorType["n_ground_truth_components"] = field(init=False)
    corr_matrix: Optional[
        TensorType["n_ground_truth_components", "n_ground_truth_components"]
    ] = field(init=False)
    component_probs: Optional[TensorType["n_ground_truth_components"]] = field(init=False)

    def __post_init__(self):
        self.frac_nonzero = self.feature_num_nonzero / self.n_ground_truth_components

        # Define the probabilities of each component being included in the data
        self.decay = torch.tensor(
            [self.feature_prob_decay**i for i in range(self.n_ground_truth_components)]
        ).to(self.device)  # FIXME: 1 / i

        self.component_probs = self.decay * self.frac_nonzero  # Only if non-correlated
        if self.feats is None:
            self.feats = generate_rand_feats(
                self.activation_dim,
                self.n_ground_truth_components,
                device=self.device,
            )

    def send(self, ignored_arg: Any) -> TensorType["dataset_size", "activation_dim"]:
        torch.manual_seed(self.generated_so_far)  # Set a deterministic seed for reproducibility
        self.generated_so_far += 1

        # Assuming generate_rand_dataset is your data generation function
        _, ground_truth, data = generate_rand_dataset(
            self.n_ground_truth_components,
            self.batch_size,
            self.component_probs,
            self.feats,
            self.device,
        )
        return ground_truth, data

    def throw(self, type: Any = None, value: Any = None, traceback: Any = None) -> None:
        raise StopIteration


def generate_rand_dataset(
    n_ground_truth_components: int,  #
    dataset_size: int,
    feature_probs: TensorType["n_ground_truth_components"],
    feats: TensorType["n_ground_truth_components", "activation_dim"],
    device: Union[torch.device, str],
) -> Tuple[
    TensorType["n_ground_truth_components", "activation_dim"],
    TensorType["dataset_size", "n_ground_truth_components"],
    TensorType["dataset_size", "activation_dim"],
]:
    # generate random feature strengths
    feature_strengths = torch.rand((dataset_size, n_ground_truth_components), device=device)
    # only some features are activated, chosen at random
    dataset_thresh = torch.rand(dataset_size, n_ground_truth_components, device=device)
    data_zero = torch.zeros_like(dataset_thresh, device=device)

    dataset_codes = torch.where(
        dataset_thresh <= feature_probs,
        feature_strengths,
        data_zero,
    )  # dim: dataset_size x n_ground_truth_components

    dataset = dataset_codes @ feats

    return feats, dataset_codes, dataset


def generate_rand_feats(
    feat_dim: int,
    num_feats: int,
    device: Union[torch.device, str],
) -> TensorType["n_ground_truth_components", "activation_dim"]:
    data_path = os.path.join(os.getcwd(), "data")
    data_filename = os.path.join(data_path, f"feats_{feat_dim}_{num_feats}.npy")

    feats = np.random.multivariate_normal(np.zeros(feat_dim), np.eye(feat_dim), size=num_feats)
    feats = feats.T / np.linalg.norm(feats, axis=1)
    feats = feats.T

    feats_tensor = torch.from_numpy(feats).to(device).float()
    return feats_tensor


# AutoEncoder Definition
class AutoEncoder(nn.Module):
    def __init__(self, activation_size, n_dict_components):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(nn.Linear(activation_size, n_dict_components), nn.ReLU())
        self.decoder = nn.Linear(n_dict_components, activation_size, bias=False)

        # Initialize the decoder weights orthogonally
        nn.init.orthogonal_(self.decoder.weight)

    def forward(self, x):
        c = self.encoder(x)

        # Apply unit norm constraint to the decoder weights
        self.decoder.weight.data = nn.functional.normalize(self.decoder.weight.data, dim=0)

        x_hat = self.decoder(c)
        return x_hat, c

    def get_dictionary(self):
        self.decoder.weight.data = nn.functional.normalize(self.decoder.weight.data, dim=0)
        return self.decoder.weight

    @property
    def device(self):
        return next(self.parameters()).device


def cosine_sim(
    vecs1: Union[torch.Tensor, torch.nn.parameter.Parameter, npt.NDArray],
    vecs2: Union[torch.Tensor, torch.nn.parameter.Parameter, npt.NDArray],
) -> np.ndarray:
    vecs = [vecs1, vecs2]
    for i in range(len(vecs)):
        if not isinstance(vecs[i], np.ndarray):
            vecs[i] = vecs[i].detach().cpu().numpy()  # type: ignore
    vecs1, vecs2 = vecs
    normalize = lambda v: (v.T / np.linalg.norm(v, axis=1)).T
    vecs1_norm = normalize(vecs1)
    vecs2_norm = normalize(vecs2)

    return vecs1_norm @ vecs2_norm.T


def mean_max_cosine_similarity(ground_truth_features, learned_dictionary, debug=False):
    # Calculate cosine similarity between all pairs of ground truth and learned features
    cos_sim = cosine_sim(ground_truth_features, learned_dictionary)
    # Find the maximum cosine similarity for each ground truth feature, then average
    mmcs = cos_sim.max(axis=1).mean()
    return mmcs


def calculate_mmcs(auto_encoder, ground_truth_features):
    learned_dictionary = auto_encoder.decoder.weight.data.t()
    with torch.no_grad():
        mmcs = mean_max_cosine_similarity(
            ground_truth_features.to(auto_encoder.device), learned_dictionary
        )
    return mmcs


def get_alive_neurons(auto_encoder, data_generator, n_batches=10):
    """
    :param result_dict: dictionary containing the results of a single run
    :return: number of dead neurons

    Estimates the number of dead neurons in the network by running a few batches of data through the network and
    calculating the mean activation of each neuron. If the mean activation is 0 for a neuron, it is considered dead.
    """
    outputs = []
    for i in range(n_batches):
        ground_truth, batch = next(data_generator)
        x_hat, c = auto_encoder(
            batch
        )  # x_hat: (batch_size, activation_dim), c: (batch_size, n_dict_components)
        outputs.append(c)
    outputs = torch.cat(outputs)  # (n_batches * batch_size, n_dict_components)
    mean_activations = outputs.mean(
        dim=0
    )  # (n_dict_components), c is after the ReLU, no need to take abs
    alive_neurons = mean_activations > 0
    return alive_neurons

In [ ]:
cfg = ToyArgs()
cfg.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(cfg.seed)
np.random.seed(cfg.seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Loading Saved SAEs

In [ ]:
# load saes
seed_string = f"{cfg.seed}" if cfg.seed >= 0 else ""
print(f"loading from toy_saes{seed_string}")
auto_encoders = [[None for l1_coef in cfg.lp_alphas] for p in cfg.ps]
for p_id, p in enumerate(cfg.ps):
    for lp_id, lp_alpha in enumerate(cfg.lp_alphas):
        save_name = f"sae_l{p}_{lp_alpha}"
        auto_encoder = torch.load(
            f"/root/sparsify/trained_models/toy_saes{seed_string}/{save_name}.pt"
        )
        auto_encoders[p_id][lp_id] = auto_encoder

# load ground truth features
ground_truth_features = torch.load(
    f"/root/sparsify/trained_models/toy_saes{seed_string}/ground_truth_features.pt"
)
data_generator = RandomDatasetGenerator(
    activation_dim=cfg.activation_dim,
    n_ground_truth_components=cfg.n_ground_truth_components,
    batch_size=cfg.batch_size,
    feature_num_nonzero=cfg.feature_num_nonzero,
    feature_prob_decay=cfg.feature_prob_decay,
    correlated=cfg.correlated_components,
    device=device,
    generated_so_far=cfg.seed,
    feats=ground_truth_features,
)

In [ ]:
# recompute losses
final_reconstruction = torch.zeros((len(cfg.ps), len(cfg.lp_alphas)))
final_lp = torch.zeros((len(cfg.ps), len(cfg.lp_alphas)))

for epoch in tqdm(range(1)):
    ground_truth, batch = next(data_generator)

    for p_id, p in enumerate(cfg.ps):
        for lp_id, lp_alpha in enumerate(cfg.lp_alphas):
            with torch.no_grad():
                auto_encoder = auto_encoders[p_id][lp_id]

                # Forward pass
                x_hat, c = auto_encoder(batch)

                # Compute the reconstruction loss and L1 regularization
                l_reconstruction = torch.nn.MSELoss()(batch, x_hat)
                l_lp = lp_alpha * torch.norm(c, p, dim=1).mean() / c.size(1)

                final_reconstruction[p_id, lp_id] = l_reconstruction.detach().cpu().clone()
                final_lp[p_id, lp_id] = l_lp.detach().cpu().clone()

In [ ]:
def effective_rank(matrix: torch.Tensor) -> torch.Tensor:
    # For each row of a matrix, return the effective count of the non-zero values.
    # If all the nonzero values are the same, this equals the number of nonzero values

    matrix = torch.clamp(matrix, min=0.0)  # should be unnecessary
    normalized_matrix = (matrix / matrix.sum(dim=-1, keepdim=True)).nan_to_num(0)
    plogp = (normalized_matrix * normalized_matrix.log()).nan_to_num(0)
    shannon_entropy = -plogp.sum(dim=-1)
    return shannon_entropy.exp()


def centroid(matrix: torch.Tensor) -> torch.Tensor:
    # For each row of a matrix, return the effective count of the non-zero values.
    # If all the nonzero values are the same, this equals the number of nonzero values

    matrix = torch.clamp(matrix, min=0.0)  # should be unnecessary
    matrix, indices = torch.sort(matrix, dim=1, descending=True)
    # normalized_matrix = (matrix / matrix.sum(dim=-1, keepdim=True)).nan_to_num(0)
    positions = torch.arange(matrix.shape[1], device=matrix.device).unsqueeze(0)
    centroid = (matrix * positions).sum(dim=-1, keepdim=True) / matrix.sum(dim=-1, keepdim=True)
    return centroid.nan_to_num(0) + 1


def count_top(matrix: torch.Tensor, threshold=0.5) -> torch.Tensor:
    # For each row of a matrix, return the number of values greater than threshold times max

    matrix = torch.clamp(matrix, min=0.0)  # should be unnecessary
    max_sim = matrix.max(dim=-1, keepdim=True)[0]

    return (matrix >= max_sim * threshold).sum(dim=-1, keepdim=True)


def effective_count(matrix: torch.Tensor) -> torch.Tensor:
    return count_top(matrix)


def similarity_concentration_oneway(matrix, filter_dead_rows=False):
    # half of the full metric, using only one direction
    # (either just monosemanticity or superposition checking)
    max_sim = matrix.max(dim=-1, keepdim=True)[0]
    eff_feature_count = effective_count(matrix)
    rowwise_concentration = max_sim / eff_feature_count
    if filter_dead_rows:
        rowwise_concentration = rowwise_concentration[max_sim > 0]
    return rowwise_concentration.mean()


def similarity_concentration(matrix, ground=None, empirical=None, alive_neurons=None):
    """
    Compute the similarity_concentration metric given:
        matrix[ground_feature, sae_feature]: number of co-occurrences, or cosine similarities
        ground[ground_feature]: total number of appearances of ground features, or None if cosine_sims
        empirical[sae_feature]: total number of appearances of empirical features, or None if cosine_sims
    method: (max(M)/effective_count(M)) for both matrix and matrix.mT, then combined
    """

    if ground is None:
        ground = torch.ones_like(matrix[:, 0])
    if empirical is None:
        empirical = torch.ones_like(matrix[0, :])

    if alive_neurons is None:
        alive_neurons = torch.full_like(matrix[0], True)

    matrix = matrix[:, alive_neurons]
    empirical = empirical[alive_neurons]

    ground_truth_sc = similarity_concentration_oneway(matrix / ground[:, None])
    sae_feature_sc = similarity_concentration_oneway(
        matrix.mT / empirical[:, None], filter_dead_rows=True
    )

    return (ground_truth_sc * sae_feature_sc).sqrt(), ground_truth_sc, sae_feature_sc  # geo mean


class BatchCorrelationCalculator:
    def __init__(self, feature_dim_x, feature_dim_y, device="cpu"):
        # Initialize sums needed for correlation calculation for each feature dimension
        self.sum_x = torch.zeros(feature_dim_x, device=device)
        self.sum_y = torch.zeros(feature_dim_y, device=device)
        self.sum_x2 = torch.zeros(feature_dim_x, device=device)
        self.sum_y2 = torch.zeros(feature_dim_y, device=device)
        self.sum_xy = torch.zeros(
            (feature_dim_x, feature_dim_y), device=device
        )  # This now becomes a matrix
        self.n = 0
        self.device = device

    def update(self, x_batch, y_batch):
        # Ensure input batches are torch tensors
        x_batch = x_batch.to(self.device)
        y_batch = y_batch.to(self.device)

        # Update running sums with a new batch
        self.sum_x += torch.sum(x_batch, dim=0)
        self.sum_y += torch.sum(y_batch, dim=0)
        self.sum_x2 += torch.sum(x_batch**2, dim=0)
        self.sum_y2 += torch.sum(y_batch**2, dim=0)
        for i in range(x_batch.shape[1]):  # Iterate over features in x
            for j in range(y_batch.shape[1]):  # Iterate over features in y
                self.sum_xy[i, j] += torch.sum(x_batch[:, i] * y_batch[:, j])
        self.n += x_batch.shape[0]

    def compute_correlation(self):
        # Compute Pearson correlation coefficient matrix between features of the two vectors
        numerator = self.n * self.sum_xy - torch.ger(self.sum_x, self.sum_y)
        denominator = torch.sqrt(
            torch.ger(self.n * self.sum_x2 - self.sum_x**2, self.n * self.sum_y2 - self.sum_y**2)
        )

        # Handle division by zero for cases with no variance
        valid = denominator != 0
        correlation_matrix = torch.zeros_like(denominator)
        correlation_matrix[valid] = numerator[valid] / denominator[valid]

        # Set correlations to 0 where denominator is 0 (indicating no variance)
        correlation_matrix[~valid] = 0

        return correlation_matrix

In [ ]:
# compute similarity concentration for each sae
sim_conc_co_occur = torch.zeros(
    (len(cfg.ps), len(cfg.lp_alphas), 3)
)  # p, lp_alpha, ground or sae sc
sim_conc_cosim = torch.zeros((len(cfg.ps), len(cfg.lp_alphas), 3))  # p, lp_alpha, ground or sae sc
sim_conc_corr = torch.zeros((len(cfg.ps), len(cfg.lp_alphas), 3))  # p, lp_alpha, ground or sae sc
mmcs = torch.zeros((len(cfg.ps), len(cfg.lp_alphas)))
features_per_ground = torch.zeros((len(cfg.ps), len(cfg.lp_alphas)))

for p_id, p in enumerate(cfg.ps):
    for lp_id, lp_alpha in enumerate(cfg.lp_alphas):
        auto_encoder = auto_encoders[p_id][lp_id]

        # adjacency_matrix[ground_truth_feature,sae_feature] = number of times they co-occur
        adjacency_matrix = torch.zeros(
            (cfg.n_ground_truth_components, cfg.n_components_dictionary), device=device
        )

        # iteratively calculate correlations
        corr_calculator = BatchCorrelationCalculator(
            cfg.n_ground_truth_components, cfg.n_components_dictionary, device=cfg.device
        )
        
        # feat_corr_calculator = BatchCorrelationCalculator(
        #     cfg.n_components_dictionary, cfg.n_components_dictionary, device=cfg.device
        # )

        cosim_matrix = data_generator.feats @ nn.functional.normalize(
            auto_encoder.decoder.weight.data, dim=0
        )  # of shape (ground_feats, sae_feats)
        mmcs[p_id, lp_id] = cosim_matrix.max(dim=-1)[0].mean()
        # might need to flip data_generator.feats?
        ground_truth_appearances = torch.zeros((cfg.n_ground_truth_components), device=device)
        sae_appearances = torch.zeros((cfg.n_components_dictionary), device=device)

        # Filter out dead neurons: either do it out here, or inside the metric
        alive = get_alive_neurons(auto_encoder, data_generator)

        for epoch in range(1):  # range(cfg.epochs):
            ground_truth, batch = next(data_generator)

            # Forward pass
            with torch.no_grad():
                x_hat1, c1 = auto_encoder(batch)
                # x_hat2, c2 = auto_encoder2(Mbatch)

                ground_truth[ground_truth != 0] = 1
                c1[c1 != 0] = 1

                adjacency_matrix += torch.einsum("bg, bs -> gs", ground_truth, c1)
                corr_calculator.update(ground_truth, c1)
                # feat_corr_calculator.update(c1, c1)
                # equiv to ground_truth.mT @ c1
                ground_truth_appearances += ground_truth.sum(dim=0)
                sae_appearances += c1.sum(dim=0)

        sc, gsc, ssc = similarity_concentration(
            adjacency_matrix, ground_truth_appearances, sae_appearances, alive_neurons=alive
        )
        sim_conc_co_occur[p_id, lp_id, :] = torch.tensor([sc, gsc, ssc])

        sc, gsc, ssc = similarity_concentration(cosim_matrix, alive_neurons=alive)
        sim_conc_cosim[p_id, lp_id, :] = torch.tensor([sc, gsc, ssc])

        corr_matrix = corr_calculator.compute_correlation()
        sc, gsc, ssc = similarity_concentration(corr_matrix, alive_neurons=alive)
        sim_conc_corr[p_id, lp_id, :] = torch.tensor([sc, gsc, ssc])

        features_per_ground[p_id, lp_id] = (adjacency_matrix > 0).sum(dim=-1).mean(dtype=float)

In [ ]:
plt.imshow(final_reconstruction.log10())  # cosim_matrix.detach().cpu()
plt.colorbar()
plt.xlabel("Lp Coefficient")
plt.ylabel("p norm")
plt.title("Reconstruction Loss")

plt.xticks(range(len(cfg.lp_alphas)), cfg.lp_alphas, rotation=30)
plt.yticks(
    range(len(cfg.ps)),
    cfg.ps,
)

In [ ]:
plt.imshow(features_per_ground)  # cosim_matrix.detach().cpu()
plt.colorbar()
plt.xlabel("Lp Coefficient")
plt.ylabel("p norm")
plt.title("Number of Active Features")

plt.xticks(range(len(cfg.lp_alphas)), cfg.lp_alphas, rotation=30)
plt.yticks(
    range(len(cfg.ps)),
    cfg.ps,
)

In [ ]:
p_id = 7
l1_id = 10
cosim_matrix = (
    (
        data_generator.feats
        @ nn.functional.normalize(auto_encoders[p_id][l1_id].decoder.weight.data, dim=0)
    )
    .detach()
    .cpu()
)
sorted_cosim_matrix = cosim_matrix.sort(descending=True, dim=-1)[0].cpu()
plt.imshow(sorted_cosim_matrix)  # cosim_matrix.detach().cpu()
plt.colorbar()
plt.xlabel("Sorted SAE features")
plt.ylabel("Ground truth features")
plt.title(f"Sorted Cosim matrix for p {cfg.ps[p_id]}, lp_alpha {cfg.lp_alphas[l1_id]}")

# plt.xticks(range(len(cfg.lp_alphas)), cfg.lp_alphas, rotation=30)
# plt.yticks(
#     range(len(cfg.ps)),
#     cfg.ps,
# )

In [ ]:
sorted_cosim = cosim_matrix[0].clamp(min=0).sort(descending=True)[0].cpu()
threshold = 0.5
plt.plot(sorted_cosim)
plt.axvline(effective_rank(cosim_matrix[0:1]).cpu().item(), label="Effective Rank", color="black")
plt.axvline(centroid(cosim_matrix[0:1]).cpu().item(), label="Center of Mass", color="brown")
plt.axvline(
    count_top(cosim_matrix[0:1], threshold=0.5).cpu().item(),
    label=f">{threshold}*max",
    color="green",
)
plt.title("Different Effective Count Methods")
plt.xlabel("Sorted Features")
plt.ylabel("Cosine Similarity to True Feature 0")
plt.legend()

In [ ]:
# use either sim_conc_cosim or sim_conc_co_occur
# plt.imshow(sim_conc_cosim[...,0]) #.mean(dim=-1))
plt.imshow(mmcs)
plt.colorbar()
plt.xlabel("Lp Coefficient")
plt.ylabel("p norm")
plt.title("MMCS")

plt.xticks(range(len(cfg.lp_alphas)), cfg.lp_alphas, rotation=30)
plt.yticks(
    range(len(cfg.ps)),
    cfg.ps,
)

In [ ]:
# use either sim_conc_cosim or sim_conc_co_occur
# plt.imshow(sim_conc_cosim[...,0]) #.mean(dim=-1))
plt.imshow(sim_conc_cosim[..., 0])
plt.colorbar()
plt.xlabel("Lp Coefficient")
plt.ylabel("p norm")
plt.title("Similarity Concentration (Using Cosine-sim Matrix)")

plt.xticks(range(len(cfg.lp_alphas)), cfg.lp_alphas, rotation=30)
plt.yticks(
    range(len(cfg.ps)),
    cfg.ps,
)

In [ ]:
# use either sim_conc_cosim or sim_conc_co_occur
# plt.imshow(sim_conc_cosim[...,0]) #.mean(dim=-1))
plt.imshow(sim_conc_co_occur[..., 0])
plt.colorbar()
plt.xlabel("Lp Coefficient")
plt.ylabel("p norm")
plt.title("Similarity Concentration (Using Co-occurence Matrix)")

plt.xticks(range(len(cfg.lp_alphas)), cfg.lp_alphas, rotation=30)
plt.yticks(
    range(len(cfg.ps)),
    cfg.ps,
)

In [ ]:
# use either sim_conc_cosim or sim_conc_co_occur
# plt.imshow(sim_conc_cosim[...,0]) #.mean(dim=-1))
plt.imshow(sim_conc_corr[..., 0])
plt.colorbar()
plt.xlabel("Lp Coefficient")
plt.ylabel("p norm")
plt.title("Similarity Concentration (Using Correlation Matrix)")

plt.xticks(range(len(cfg.lp_alphas)), cfg.lp_alphas, rotation=30)
plt.yticks(
    range(len(cfg.ps)),
    cfg.ps,
)

# Examining just the L1